In [1]:
import pandas as pd
import json
from geopy import distance

In [ ]:
with open('data 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.columns


In [2]:
df = pd.read_csv('../data/new_addresses.csv')
df.head()

,Casenumber,address
0,ME2021-03873,15300 West Avenue ORLAND PARK 60462
1,ME2021-03868,6543 S KAMINSKI CHICAGO 60629
2,ME2021-03864,15234 Paulina HARVEY 60426
3,ME2021-03859,832 N. MOZART ST. CHICAGO 60622
4,ME2021-03858,939 JACKSON CIRCLE ELK GROVE VILLAGE 60007


In [17]:
def add_state(x: str) -> str:
    words = x.split()
    words.insert(-1, 'Illinois')
    return ' '.join(words)

df['address'] = df.address.apply(lambda x: add_state(x))
df.address.head()

0         15300 West Avenue ORLAND PARK Illinois 60462
1               6543 S KAMINSKI CHICAGO Illinois 60629
2                  15234 Paulina HARVEY Illinois 60426
3             832 N. MOZART ST. CHICAGO Illinois 60622
4    939 JACKSON CIRCLE ELK GROVE VILLAGE Illinois ...
Name: address, dtype: object

In [5]:
all((a for a in df.address.values if a.split()[-1].isdigit()))

True

In [16]:
w = df.address.values[5].split()
w.insert(-1, 'Illinois')
w

['6135', 'N.', 'KEATING', 'AVE', 'CHICAGO', 'Illinois', '60646']

In [18]:
df.to_csv('new_addresses.csv', index=False)

In [5]:
df1 = pd.read_csv('../data/me_geocoding.csv')
df1.drop('id', axis=1, inplace=True)
df1.columns = ('Casenumber', 'address', 'long', 'lat', 'rating')
print(df1.columns)
df1.sample(2)

Index(['Casenumber', 'address', 'long', 'lat', 'rating'], dtype='object')


,Casenumber,address,long,lat,rating
15348,ME2021-01412,21ST AND ST. CHARLES MAYWOOD Illinois 60153,-87.759671,41.068281,71.0
48873,ME2017-02606,11612 South Ridgeland Ave WORTH Illinois 60482,-87.778519,41.679718,0.0


In [6]:
df2 = pd.read_csv('../data/CookCountyMEArchives.csv')
print(df2.columns)
df2.sample(2)

Index(['Age', 'Casenumber', 'Cold Related', 'Commissioner District',
       'Death Date', 'Gender', 'Gunrelated', 'Heat Related', 'Incident City',
       'Incident Date', 'Incident Street', 'Incident Zip', 'Latino',
       'Latitude', 'Longitude', 'Manner', 'Opioids', 'Primarycause',
       'Primarycause Linea', 'Race', 'Residence City', 'Residence Zip',
       'Secondarycause'],
      dtype='object')
/Users/nick/miniforge3/envs/geo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Age,Casenumber,Cold Related,Commissioner District,Death Date,Gender,Gunrelated,Heat Related,Incident City,Incident Date,...,Latitude,Longitude,Manner,Opioids,Primarycause,Primarycause Linea,Race,Residence City,Residence Zip,Secondarycause
50057,56.0,ME2020-05462,False,7.0,2020-05-13T15:15:00.000,Male,False,False,CHICAGO,2020-05-13T15:00:00.000,...,41.80605,-87.663757,NATURAL,False,DIABETIC KETOACIDOSIS,DIABETES MELLITUS,White,Chicago,60609.0,HYPERTENSION
26068,61.0,ME2020-04915,False,NaN,2020-05-08T03:04:00.000,Male,False,False,DES PLAINES,2020-04-20T12:00:00.000,...,NaN,NaN,NATURAL,False,PNEUMONIA,NOVEL CORONA (COVID-19) VIRUS INFECTION,White,Des Plaines,60016,"AORTIC STENOSIS, HYPOTHYROIDISM"


In [22]:
import numpy as np

def calc_feet(x: float):
    return x * 5_280

def calc_distance(row):
    if pd.isna(row.lat) or pd.isna(row.long) or pd.isna(row.Latitude) or pd.isna(row.Longitude):
        return np.nan  # return nan if any nulls
    else:
        m = distance.distance((row.lat, row.long), (row.Latitude, row.Longitude))
        feet = calc_feet(m.miles)
        return feet


In [23]:
df3 = df1.set_index('Casenumber').join(df2.set_index('Casenumber'), on='Casenumber')
df3['distance'] = df3.apply(lambda row: calc_distance(row), axis=1)

In [30]:
len(df1[pd.notna(df1.long)])

50018

In [31]:
df3['distance'].describe().round(2)

count      44639.00
mean       27434.87
std       124475.93
min            3.32
25%           73.53
50%          188.27
75%         7839.15
max      1753597.17
Name: distance, dtype: float64

In [32]:
df3.distance.corr(df3.rating).round(3)

0.241